In [2]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline 
from functools import reduce

In [3]:
import pyarrow

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

ImportError: numpy.core.multiarray failed to import

In [5]:
import sys
!"{sys.executable}" -m pip install numpy --upgrade

  Found existing installation: numpy 1.11.1
    Uninstalling numpy-1.11.1:
      Successfully uninstalled numpy-1.11.1


Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\PPITER~1\\AppData\\Local\\Temp\\pip-uninstall-1t7sb0iz\\users\\ppitera002\\appdata\\local\\continuum\\anaconda3\\lib\\site-packages\\numpy\\core\\multiarray.cp35-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
app_train = pd.read_csv('../1_data_import/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
bureau = pd.read_csv('../1_data_import/bureau.csv')
print('Bureau shape: ', bureau.shape)
bureau.head()

In [ ]:
bureau["CREDIT_ACTIVE_NUM"] = np.where(bureau["CREDIT_ACTIVE"]=="Closed",0,1)
bureau.sort(columns=["SK_ID_CURR","SK_ID_BUREAU"])

In [ ]:
#bureau_sample = bureau.head(100000)
bureau_max = bureau.groupby("SK_ID_CURR").max()
bureau_max.columns = ["ALL_MAX_"+x for x in bureau_max.columns]
bureau_count = bureau.groupby("SK_ID_CURR").count()
bureau_count.columns = ["ALL_COUNT_"+x for x in bureau_count.columns]
bureau_mean = bureau.groupby("SK_ID_CURR").mean()
bureau_mean.columns = ["ALL_MEAN_"+x for x in bureau_mean.columns]
bureau_sum = bureau.groupby("SK_ID_CURR").sum()
bureau_sum.columns = ["ALL_SUM_"+x for x in bureau_sum.columns]

In [ ]:
bureau_max = bureau_max.reset_index()
bureau_count = bureau_count.reset_index()
bureau_mean = bureau_mean.reset_index()
bureau_sum = bureau_sum.reset_index()

In [ ]:
bureau_balance = pd.read_csv('../1_data_import/bureau_balance.csv')
bureau_balance_last_active = bureau_balance[bureau_balance.STATUS!='C'].groupby('SK_ID_BUREAU').max().reset_index()

In [ ]:
bureau_from_last_12M = pd.merge(bureau
                                ,bureau_balance_last_active
                                ,how="left"
                                ,on = "SK_ID_BUREAU"
                               )
bureau_from_last_12M = bureau_from_last_12M[bureau_from_last_12M.MONTHS_BALANCE>=-12]

In [ ]:
#bureau_sample = bureau.head(100000)
bureau_max_from_last_12M = bureau_from_last_12M.groupby("SK_ID_CURR").max()
bureau_max_from_last_12M.columns = ["12M_MAX_"+x for x in bureau_max_from_last_12M.columns]
bureau_count_from_last_12M = bureau_from_last_12M.groupby("SK_ID_CURR").count()
bureau_count_from_last_12M.columns = ["12M_COUNT_"+x for x in bureau_count_from_last_12M.columns]
bureau_mean_from_last_12M = bureau_from_last_12M.groupby("SK_ID_CURR").mean()
bureau_mean_from_last_12M.columns = ["12M_MEAN_"+x for x in bureau_mean_from_last_12M.columns]
bureau_sum_from_last_12M = bureau_from_last_12M.groupby("SK_ID_CURR").sum()
bureau_sum_from_last_12M.columns = ["12M_SUM_"+x for x in bureau_sum_from_last_12M.columns]

In [ ]:
bureau_max_from_last_12M = bureau_max_from_last_12M.reset_index()
bureau_count_from_last_12M = bureau_count_from_last_12M.reset_index()
bureau_mean_from_last_12M = bureau_mean_from_last_12M.reset_index()
bureau_sum_from_last_12M = bureau_sum_from_last_12M.reset_index()

In [ ]:
dfs = [bureau_max,bureau_count,bureau_max_from_last_12M,bureau_count_from_last_12M,bureau_mean_from_last_12M,bureau_sum_from_last_12M,bureau_mean,bureau_sum]

In [ ]:
bureau_final = reduce(lambda left,right: pd.merge(left,right,on='SK_ID_CURR'), dfs)

In [ ]:
#bureau_final.to_csv('bureau_features.csv')
feather.write_feather(bureau_final, 'bureau_features')